# *Classification et extraction d’information d'un document*
# **Rendu 1 : Rapport d’exploration, de data visualisation et de pre-processing des données**

***

## **Introduction au projet**

### **Contexte**

En tant que Data Scientist, le projet OCR s'inscrit dans le cadre de l'analyse et du traitement automatisé des données textuelles. L'objectif est de développer des modèles permettant de convertir des documents manuscrits en données exploitables, facilitant ainsi la prise de décision basée sur des informations précises et structurées.<br><br>

On appliquera des méthodes de ML et Deep Learning pour l'extraction des informations textuelles et la classification des images. Nous allons utiliser deux approches, que nous confronterons par la suite avec un algorithme de votes et de boosting :
- Sélection d'un Modèle d'OCR complété d'un modèle de NLP (performe mieux s'il y a beaucoup de texte)
- Création d'un Modèle de CNN (performe mieux s'il y a peu de texte)

### **Objectifs**

L'objectif de ce projet est de développer un ensemble de solutions pour améliorer la gestion et le traitement des documents dans le domaine des assurances.<br><br>

1. **Développer un modèle de classification des documents** : Identifier la nature des documents (contrats, justificatifs, offres, etc.) en utilisant des techniques de classification.
2. **Améliorer l'efficacité du traitement des documents** : Automatiser les tâches manuelles dans le domaine des assurances pour améliorer l'efficacité du traitement des documents.
3. **Réduire les coûts opérationnels** : Diminuer les coûts associés à la gestion des documents papier par l'automatisation des processus.<br><br>

Les membres de l'équipe :
- **Quentin Pizenberg** 
- **Julien Thielleux** 
- **Sofiane Louiba** 
- **Chaima Haddoudi**<br><br>

Aucun expert métier n'a été contacté jusqu'à présent, car cela ne semble pas nécessaire pour ce projet qui n'a pas besoin d'expertise forte dans le domaine. Cependant, il pourrait être utile de consulter des Data Scientists ayant travaillé sur des modèles d'analyses d'images similaires. Par ailleurs, aucun projet similaire n'a été identifié dans les entreprises ou dans l'entourage de l'équipe. Par conséquent, le projet actuel ne bénéficie pas d'un précédent.

## **Compréhension et manipulation des données (hors visualisation)**

### **Cadre**

La première étape de la modélisation du problème a été de formaliser et comprendre la structure
de données (qui ont été obtenues librement sur internet).<br><br>

Le dataset [RVL-CDIP](https://www.google.com/url?q=https%3A%2F%2Fadamharley.com%2Frvl-cdip%2F) contient :
- 400.000 images au format *.tif*
- 400.000 fichiers text avec le même nom que le fichier image, contenant les 16 catégories<br><br>

Le dataset [text-extraction-for-ocr de Kaggle](https://www.google.com/url?q=https%3A%2F%2Fwww.kaggle.com%2Fdatasets%2Fmanishthem%2Ftext-extraction-for-ocr%2Fdata) contient :
- 520 images au format *.tif*
- 520 fichiers xml gt correspondant aux zones de detection de l'OCR
- 520 fichiers xml ocr correspondant au contenu detecté par l'OCR de référence. Cela sera notre cible pour selectionner un OCR <br><br>

Dans la suite de la visualisation, on a pris un enxemble de 1000 fichiers du dataset RVL-CDIP selectionné aléatoirement avec une equirépartition entre chacune des catégories.

### **Pertinence**

Le projet repose sur un jeu de données peu traditionnel, composé d’images de documents plutôt que des ensembles de données textuelles ou numériques couramment analysés en machine learning.<br>
Les variables les plus pertinentes pour ce projet sont les images des documents, les annotations de classification et les champs textuels à extraire. La variable cible est la classe du document pour la classification et le texte extrait pour l’OCR.<br><br>Le jeu de données se distingue par la diversité des types de documents et la qualité variable des scans, incluant des manuscrits et des impressions.<br> Toutefois, des limitations existent, notamment en raison de la qualité des scans et des variations d’écriture manuscrite, qui peuvent poser des défis supplémentaires.

### **Pre-processing et feature engineering**

Le preprocessing est essentiel pour préparer les jeux de données, en particulier pour les images de documents.<br><br>
En améliorant la qualité des images et en extrayant des caractéristiques pertinentes, nous optimisons les performances des modèles de machine learning. Ce pré-traitement permet d'uniformiser les données, de réduire le bruit et de faciliter l'extraction d'informations, conduisant ainsi à une classification plus précise et à une extraction de texte plus fiable.<br><br>
Pour garantir une reconnaissance optimale, il est crucial que les caractères soient bien distincts du fond, avec des bordures nettes et un contraste élevé. De plus, un bon alignement des caractères et une résolution adéquate sont indispensables. Enfin, réduire le bruit des images améliore encore la qualité du traitement et des résultats obtenus.<br><br>

**Librairies utilisées :** 
- OpenCV as cv2<br>
- ImageOps from PIL<br><br>

#### **Format des Images**
- **Redimensionnement** *(img_resized = cv2.resize(img, (800, 1000)))* :
    Uniformisation des tailles des images pour standardiser l'entrée des modèles. Les images sont redimensionnées en 800x1000 pixels.
- **Rotation / Inclinaison** *(cv2.getRotationMatrix2D(center, angle, 1.0) + cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE))* : 
    Correction de l'orientation des images pour aligner les textes horizontalement. Cette opération utilise une matrice de rotation pour redresser les images inclinées, améliorant ainsi la lisibilité et la précision de l'extraction de texte.<br><br>

#### **Texte des images**
Pas besoin de resize le texte car il est assez homogène et c'est mieux d'avoir quelques différences légères pour entrainer le modèle.
- **Thinning / Skeletonization** (skeleton = cv2.ximgproc.thinning(img, thinningType=cv2.ximgproc.THINNING_ZHANGSUEN)) : Réduction de l'épaisseur des traits pour extraire une version simplifiée des éléments visuels, souvent utilisée pour les textes manuscrits. Cette technique permet d'obtenir une représentation minimale tout en conservant les caractéristiques essentielles.
<br><br>

#### **Couleurs**
Pas besoin de faire de la binarization / grayscaling car les images sont déjà en noir et blanc avec un contraste et une luminosité plutôt bons. On risquerait de perdre en lisibilité si l'on change notamment le contraste pour les petites lettres.
- **Noise Removal / Reduction** (img_denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)) : Suppression des bruits pour obtenir une image plus propre en utilisant des algorithmes de débruitage. Cela est crucial pour éliminer les petits artefacts visuels et améliorer la lisibilité.
- **Black/White inversion** PIL.Imageops.invert(img) : 
    On inverse le noir et le blanc sur les images majoritairement noires pour avoir une proportion de pixels noirs inférieur à celle des pixels blancs et uniformiser le dataset

### **Visualisations et Statistiques**

Les "relations" entre nos variables n'ont pas de sens sur notre dataset.

Cependant, la visualisation est assez complète et complexe sur des datasets d'images, donc nous avons décidé d'y consacrer une nouvelle partie présentée à la suite. Aussi nous allons montrer des visualisation avant pre et post processing.

## **Data Visualisation**

Etudions en parallèle les éléments des deux datasets.

###  **KAGGLE DATASET**

Préparons une liste principale avec les images ainsi que des tableau numpy d'informations qui seront à analyser : les tailles, résolutions, distribution de couleur et luminosité. Enfin, regardons le premier élément pour avoir une idée plus précise des types d'éléments considérés.

#### **I. Analyse des formats**

##### **I.A. Distribution des Tailles d'Images**
L'analyse de la distribution des tailles de fichiers permet de comprendre la variation des tailles d'images et de décider si une normalisation de la taille est nécessaire.<br>
En fonction des résultats, nous pouvons choisir de redimensionner les images pour une uniformité.

<div>
<img src="./../data/visualization/kaggle_image_size_distribution.png" width="600" />
</div>

**Commentaires & Constat métier**
- **Observation** : La majorité des images ont une taille de fichier comprise entre 0.05 et 0.1 Mo.
- **Recommandation** : Les images semblent être de taille appropriée pour les traitements ultérieurs. 

##### **I.B. Résolutions des Images**
L'analyse des résolutions des images permet de vérifier l'homogénéité des résolutions. Si les résolutions varient beaucoup, un redimensionnement des images peut être nécessaire pour garantir la cohérence dans le traitement ultérieur.

<div>
<img src="./../data/visualization/kaggle_image_size_repartition.png" width="600" />
</div>

Les images ont toutes la même hauteur de 1000px. Cependant la largeur n'est pas uniforme.<br>
L'analyse de la distribution des largeurs des images permet de comprendre la variation des tailles d'images et de décider si une normalisation de la dimension est nécessaire.<br>

<div>
<img src="./../data/visualization/kaggle_size_repartition.png" width="600"/>
</div>

<div>
<img src="./../data/visualization/kaggle_size_example.png" width="600" title="Image la plus petite et la plus grande du dataset Kaggle."/>
</div>

**Commentaires & Constat métier**
- **Observation** : La largeur des images varie principalement entre 750 et 800 pixels, tandis que la hauteur est uniformément à 1000 pixels.
- **Recommandation** : Pour assurer une uniformité dans les dimensions des images, un redimensionnement à une résolution standard (par exemple, 800x1000 pixels) pourrait être appliqué. L'ajout de pixels neutre sur l'un des coté ne modifirait pas l'information tout en uniformisant la taille des images. Cela faciliterait les traitements et analyses subséquents.

Avec l'uniformisation de la largeur des images à 800px :

<div>
<img src="./../data/visualization/kaggle_resized_distribution.png" width="600" />
</div>

##### **I.C. Analyse de la Rotation des Images**
Les documents écrits doivent être correctement orientés pour une reconnaissance optimale. Analyser l'orientation des images permet d'identifier les images nécessitant une correction de rotation.<br><br>
On recupere l'orientation des images avec la librairie build_features<br>

<div>
<img src="./../data/visualization/kaggle_rotation_distribution.png" width="600" />
</div>


<div>
<img src="./../data/visualization/kaggle_rotation_samples.png" width="800" />
</div>

**Commentaires + Constat métier**
- **Observation** : 
    Il y a une diversité dans l'orientation des images, avec une concentration autour de 0 degré. <br>
    35 documents sur 520, soit 6,7% des images n'ont pas d'orientation. <br>Cela est vraissemblablement du à la présence de texte en position vertical et horizontal présent sur le document.<br>
    
    La grande majorité des images ont une orientation verticale.<br>
    On observe que beaucoup d'image ont une orientation entre 0° et 90°. Cependant, lorsque l'on etudie l'image, on remarque que celle ci est bien verticale.<br>
    
    L'orientation provient des autres éléments de l'image, par exemple des marges, des tableaux, des logos, de l'écriture manuscrite.
- **Recommandation** : 
    Une correction d'angle (rotation) pour aligner les textes horizontalement serait bénéfique.<br>
    L'utilisation de la détection de l'angle d'orientation et la rotation corrective en conséquence pourrait standardiser l'orientation des documents.
    On restera attentif à l'incidense de l'orientation sur le choix de l'OCR

#### **II. Analyse du texte**

##### **II.A. Analyse du Contenu Texte et Bordures**
Pour les documents écrits, il est crucial de s'assurer que le contenu texte est centré et bien contenu dans l'image. Cette analyse aide à déterminer si des ajustements de translation sont nécessaires pour recentrer le contenu.

<div>
<img src="./../data/visualization/kaggle_text_position.png" width="600" />
</div>

**Commentaires + Constat métier**
- **Observation** : La position du contenu texte est majoritairement en haut des images.
- **Recommandation** : Pour les analyses nécessitant l'extraction de texte, un recadrage automatique pour cibler les zones où le texte est principalement situé pourrait être mis en place. Cela permettrait de concentrer les efforts de traitement sur les zones d'intérêt. 

##### **II.B. Taille Moyenne des Lettres**
La taille moyenne des lettres peut révéler des informations sur le type de document (une publicité  qui differe d'un document comptable). Une taille de lettre plus petite peut nécessiter une meilleure résolution pour la reconnaissance de caractères. Selon les résultats, une augmentation de la résolution ou un zoom sur les sections de texte pourrait être envisagé.<br><br>
Un document à une taille de police abérante de 968px, soit la taille du document.<br>
Une visualisation rapide permet de voir que la bordure a été detecté comme un caractere

<div>
<img src="./../data/visualization/kaggle_text_font_size_max.png" width="300" />
</div>

On étudie la répartition des tailles de police en excluant l'image abérante

<div>
<img src="./../data/visualization/kaggle_font_size_repartition.png" width="600" />
</div>

**Commentaire & Constat Metier** 
- **Observation** : 
    On remarque que un document possede une bordure qui a été confondue avec une lettre.
    La taille moyenne des lettres est de 5.86 pixel.
    Les documents ont tous une résolution identique de 72 dpi
- **Recommandation** : 
    Une faible taille de police peut être impactant pour la detection de caractères de l'OCR. De plus, le materiel de numérisation utilisé implique une résolution faible de 72dpi. Un eventuel agrandissement des images pourra être pris en compte si la resolution n'est pas satisfaisante pour l'OCR et ne permet pas la detection de caractères.
    Les premieres évaluations ne montre pas de besoin d'agrandissement pour detecter les lettres.

##### **II.C. Nombre Moyen de Mots**
Le nombre moyen de mots dans un document peut indiquer la densité d'information. Une densité de mots élevée peut nécessiter des techniques de prétraitement spécifiques comme la segmentation du texte. Selon les résultats, un traitement spécifique pour gérer les documents denses en texte pourrait être appliqué.

<div>
<img src="./../data/visualization/kaggle_nb_word_distribution.png" width="600" />
</div>

<div>
<img src="./../data/visualization/kaggle_min_max_word_sample.png" width="600" />
</div>

On regarde si la présence de stopwords influe sur le nombre de mots detectés

<div>
<img src="./../data/visualization/kaggle_nb_word_comparison.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    Les documents sont assez uniformement distribués autour de 100 mots.
    On observe que les valeurs extremes sont du à des petits documents peu remplis ou bien illisible d'une part et à des documents comportant beaucoup de lignes écrites d'autre part. Le nombre de mot est cohérent avec ce que l'on observe en regardant les images.
    On remarque que les données detectées ne sont pas parfaite et que beaucoup de texte peut au final ne pas être detecté.

    On observe que la presence de stopwords reste faible et impacte faiblement la taille des textes.
- **Recommandation** : 
    On cherche un OCR qui detecte le texte aussi bien que la cible du dataset

##### **II.D. Langue la plus fréquente**
Afin de detecter les bons mots et ne pas supprimer d'informations pour l'entrainement du modele NLP, il est nécessaire de regarder la langue dominante qui ressort de chaque texte.<br>
Ainsi, on pourra adapter les stopwords pour eviter de supprimer des informations importantes.<br> En fonction des résultats, on adaptera la liste des mots à exclure pour affiner les analyses de contenu.<br><br>
On detecte la langue en utilisant LanguageDetector de spacy

<div>
<img src="./../data/visualization/kaggle_langue_detection.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    Une majorité des textes (97%) sont en anglais.
    Dans les textes dans d'autres langues, seul une partie du texte n'est pas en anglais, souvent des noms propres.<br><br>
- **Recommandation** : 
    On se concentrera principalement sur la selection de stopwords en anglais car cela represente la majorité des document du dataset Kaggle.
    Les documents ayant une autre langue ont un mélange entre l'anglais et une autre langue. Ces documents auront peu d'incidence sur le modele NLP.

##### **II.E. Mots les Plus Fréquents**
Identifier les mots les plus fréquents permet de comprendre le contenu principal des documents et d'éliminer les mots sans intérêt pour les analyses de texte. En fonction des résultats, une liste de mots à exclure pourrait être générée pour affiner les analyses de contenu.<br><br>
On observe une forte présence de stopwords et ponctuation qui pollue la detection de mots.<br>
On enleve ces éléments pour faire resortir les mots les plus fréquents

<div>
<img src="./../data/visualization/kaggle_top_keywords.png" width="600" />
</div>

<div>
<img src="./../data/visualization/kaggle_top_clean_keyword.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    Les stopswords sont assez fréquents et polluent la lecture des fichiers texte de facon automatique.<br>
    Après nettoyage, on remarque que le mot le plus fréquent est Invoice ce qui est cohérent avec le dataset kaggle qui ne regroupe que des factures.<br><br>
- **Recommandation** : 
    On eliminera les stopwords avant d'utiliser un modele NLP afin de ne conserver que les mots importants et facilitant la classification du document

#### **III. Analyse des couleurs**

##### **III.A. Distribution des Intensités de Couleurs**
Analyser la distribution des intensités de couleurs permet de comprendre la répartition des couleurs dans les images et d'identifier les éventuelles valeurs aberrantes. Cela aide à décider si un ajustement de la balance des couleurs ou une normalisation est nécessaire.

<div>
<img src="./../data/visualization/kaggle_intensite_color.png" width="600" />
</div>

**Commentaires & Constat métier**
- **Observation** : Une majorité des pixels ont des valeurs d'intensité élevées, proche de 250.<br>
Les images sont déja assez contrastées à la vue des pics autour de 0 et 250
- **Recommandation** : Il pourrait être bénéfique d'augmenter le contraste ou d'effectuer une égalisation d'histogramme pour les images où la majorité des pixels ont des valeurs d'intensité intermédiaires, afin d'améliorer la visibilité et la différenciation des caractéristiques.

##### **III.B. Luminosité Moyenne**
L'analyse de la luminosité moyenne des images permet de vérifier si les images ont des niveaux de luminosité cohérents. Des ajustements peuvent être nécessaires pour corriger les variations de luminosité afin d'assurer une qualité d'image uniforme.

<div>
<img src="./../data/visualization/kagggle_luminence.png" width="600" />
</div>

**Commentaires & Constat métier**
- **Observation** : La plupart des images ont une luminosité moyenne élevée, avec un pic autour de 240.
- **Recommandation** : Pour les images avec une luminosité moyenne faible, une correction de luminosité pourrait être appliquée pour améliorer la lisibilité. Pour les images très lumineuses, une réduction de la luminosité pourrait être nécessaire pour éviter la saturation.

##### **III.C. Distribution des Couleurs pour Chaque Image**
Analyser la distribution des couleurs pour chaque image permet d'identifier les variations de couleur d'une image à l'autre. Cela peut aider à déterminer si des ajustements spécifiques par image sont nécessaires pour la normalisation des couleurs.

**Commentaires + Constat métier**
- **Observation** : Les images ont toutes un même spectre de couleur.<br>
    De plus, on souhaite detecter le texte dans les images. 
    La couleur n'apporte pas d'information complémentaire
- **Recommandation** : 
    On utilisera les images passées en échelle de gris pour ne pas avoir de bruit lié à la couleur

##### **III.D. Pourcentage de Pixels Noirs dans l'Image**
Le pourcentage de pixels noirs peut indiquer la densité de numérisation ou la présence d'arrière-plans sombres et d'images importantes.<br>
Une forte densité de pixels noirs pourrait nécessiter un prétraitement pour améliorer le contraste.<br>
Selon les résultats, une augmentation du contraste, un filtrage des pixels noirs ou une inversion des images pourrait être nécessaire.

<div>
<img src="./../data/visualization/kaggle_dark_pixel_distribution.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    La majorité des images possedent une proportion de 1% à 5% de pixels sombres. Les quelques images possedant une bordure sombre explique le pic à 24% de pixel sombre.
- **Recommandation** : 
    La distribution des pixels sombres sur le dataset de Kaggle est assez uniforme et ne necessite pas de prétraitement nécéssaire pour ajuster les images

###  **KRVL_CDIP DATASET**

Le dataset initial comporte plus de 400.000 images.<br><br>
Afin de simplifier l'analyse et de pouvoir comparer facilement avec le dataset de Kaggle, nous avons séléctionné 992 images, respectivement 62 images prises aléatoirement dans chacune des 16 catégories.<br>
Préparons une liste principale avec les images ainsi que des tableau d'informations qui seront à analyser : les tailles, résolutions, distribution de couleur et luminosité. <br>
Enfin, regardons le premier élément pour avoir une idée plus précise des types d'éléments considérés.

#### **I. Analyse des formats**

##### I.A. Distribution des Dimensions d'Images
Les images ont toutes la même hauteur de 1000px. Cependant la largeur n'est pas uniforme.<br>
L'analyse de la distribution des largeurs des images permet de comprendre la variation des tailles d'images et de décider si une normalisation de la dimension est nécessaire.<br>


<div>
<img src="./../data/visualization/rvlcdip_size_repartition.png" width="600"/>
</div>

**Commentaires**
- **Observation** : 
    La largeur des images varie principalement entre 750 et 800 pixels, tandis que la hauteur est uniformément à 1000 pixels.
    On observe une repartition assez similaire à celle observé sur le dataset de Kaggle
- **Recommandation** : 
    Pour assurer une uniformité dans les dimensions des images, un redimensionnement à une résolution standard (par exemple, 800x1000 pixels) pourrait être appliqué. Le redimensionnement pourra être identique pour l'entrainement du CNN et l'entrainement de l'OCR

En redimentionnant le dataset, on uniformise la taille des images pour l'utilisation du CNN et de l'OCR qui pourraient y être sensible

<div>
<img src="./../data/visualization/rvlcdip_resize_picture.png" width="600"/>
</div>

##### I.B. Analyse de la Rotation des Images
Les documents écrits doivent être correctement orientés pour une reconnaissance optimale. Analyser l'orientation des images permet d'identifier les images nécessitant une correction de rotation.<br><br>
On recupere l'orientation des images avec la librairie build_features<br>

<div>
<img src="./../data/visualization/rvlcdip_rotation_distribution.png" width="600" />
</div>


<div>
<img src="./../data/visualization/rvlcdip_rotation_sample.png" width="800" />
</div>

**Commentaires**
- **Observation** : 
    Il y a une diversité dans l'orientation des images, avec une concentration autour de 0 degré. <br>
    Les documents sans orientation sont du à la présence de texte en position vertical et horizontal sur le document.<br><br>
    On observe une plus grande diversité des orientations sur les images, qui n'est pas biaisé par la présence de lignes verticales et de tableaux.<br><br>
- **Recommandation** : 
    Une correction d'angle pour aligner les textes horizontalement serait bénéfique pour un choix d'un OCR, cependant elle ne refleterait pas le comportement naturel pour la detection via un CNN.<br><br>
    L'utilisation de la détection de l'angle d'orientation et la rotation corrective en conséquence pourrait standardiser l'orientation des documents. On restera attentif à l'incidense de l'orientation sur le choix de l'OCR et on appliquera la meme transformation pour l'OCR et le CNN

#### II. Analyse du texte

##### II.A. Taille Moyenne des Lettres
La taille moyenne des lettres peut révéler des informations sur le type de document (une publicité  en gros caracteres contre un document comptable en petite police).<br><br> Une taille de lettre plus petite peut nécessiter une meilleure résolution pour la reconnaissance de caractères. Selon les résultats, une augmentation de la résolution ou un zoom sur les sections de texte pourrait être envisagé.<br><br>


<div>
<img src="./../data/visualization/rvlcdip_letter_size_distribution_raw.png" width="600" />
</div>

On remarque 3 valeurs abérantes. Ce sont des documents avec des tailles de caractères de plus de 800 px.<br>
Une visualisation rapide permet de voir que la bordure a été detecté comme un caractere

<div>
<img src="./../data/visualization/rvlcdip_max_letter_size.png" width="600" />
</div>

On étudie la répartition des tailles de police en excluant les images abérantes.

<div>
<img src="./../data/visualization/rvlcdip_letter_size_clean_distribution.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    On remarque que certains documents possede des bordures et des fonds qui ont été confondues avec des lettres.
    La taille moyenne des lettres est de 5.35 pixel.
    Elle reste cohérente avec la taille de pixel observé sur le dataset de Kaggle.
    Les deux datasets sont cohérents sur la résolution et la taille des caractères.
    Les documents ont tous une résolution identique de 72 dpi
- **Recommandation** : 
    On appliquera le même préprocessing sur la définition des documents sur le dataset d'entrainement de Kaggle et de RVL-CDIP pour l'OCR. 


#### III. Analyse des couleurs

##### III.A. Luminosité Moyenne
L'analyse de la luminosité moyenne des images permet de vérifier si les images ont des niveaux de luminosité cohérents. Des ajustements peuvent être nécessaires pour corriger les variations de luminosité afin d'assurer une qualité d'image uniforme.

**Commentaires**
- **Observation** : La plupart des images ont une luminosité moyenne élevée, avec un pic autour de 240.
- **Recommandation** : Pour les images avec une luminosité moyenne faible, une correction de luminosité pourrait être appliquée pour améliorer la lisibilité. Pour les images très lumineuses, une réduction de la luminosité pourrait être nécessaire pour éviter la saturation.

##### III.B. Distribution des Couleurs pour Chaque Image
Analyser la distribution des couleurs pour chaque image permet d'identifier les variations de couleur d'une image à l'autre. Cela peut aider à déterminer si des ajustements spécifiques par image sont nécessaires pour la normalisation des couleurs.

**Commentaires**
- **Observation** : Les images sont en noir et blanc.
- **Recommandation** : Aucune transformation nécéssaire.

##### III.C. Pourcentage de Pixels Noirs dans l'Image
Le pourcentage de pixels noirs peut indiquer la densité de numérisation ou la présence d'arrière-plans sombres et d'images importantes.<br>
Une forte densité de pixels noirs pourrait nécessiter un prétraitement pour améliorer le contraste.<br>

<div>
<img src="./../data/visualization/rvlcdip_dark_pixel_distribution.png" width="600" />
</div>

On remarque une distribution concentré autour d'une faible densité de pixel, avec des valeurs extremes assez distantes de la moyenne.<br>On observe en détaille un extrait des images avec une faible et une forte densité de pixels noirs.

<div>
<img src="./../data/visualization/rvlcdip_dark_pixel_sample.png" width="600" />
</div>

**Commentaire** : 
- **Observation** : 
    La majorité des images possedent une proportion de 1% à 5% de pixels sombres. 
    <br>
    Cependant, on observe des documents avec peu d'ecriture et donc une faible densité de pixels sombres et à l'inverse des images majoritairement noires.<br>
    On observe aussi sur des images sombres que le texte est écrit en blanc sur fond noir.<br><br>
- **Recommandation** : 
    Afin de reéquilibrer les données et conserver une distribution uniforme, on préconise une inversion du noir et du blanc pour les images ayant plus de 50% de pixels sombres afin de recentrer la distribution.

En inversant le noir et le blanc pour les images ayant plus de 50% de pixels noirs, on uniformise le dataset pour que les valeurs abérantes aient moins d'impacte sur l'entrainement.

<div>
<img src="./../data/visualization/rvlcdip_dark_pixel_preprocessed.png" width="600"/>
</div>